###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [1]:
import pandas as pd

###Archivo m8fd-ahd9.csv

Contiene información sobre la modalidad de hurto donde el victimario utiliza diferentes medios con el fin de apoderarse de los elementos de valor que lleva consigo una persona, se mide en víctimas.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/4rxi-8m8d.csv?$limit=600000", dtype={'cod_muni': str})
df.head()

fecha_hecho  cod_depto departamento cod_muni  \
0  2024-07-31T00:00:00.000          5    ANTIOQUIA    05615   
1  2024-07-31T00:00:00.000          5    ANTIOQUIA    05664   
2  2024-07-31T00:00:00.000          5    ANTIOQUIA    05467   
3  2024-07-31T00:00:00.000          8    ATLANTICO    08638   
4  2024-07-31T00:00:00.000          8    ATLANTICO    08421   

                   municipio  cantidad  
0                   RIONEGRO         7  
1  SAN PEDRO DE LOS MILAGROS         1  
2                 MONTEBELLO         1  
3                SABANALARGA         1  
4                    LURUACO         2

##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588117 entries, 0 to 588116
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   fecha_hecho   588117 non-null  object
 1   cod_depto     588117 non-null  int64 
 2   departamento  588117 non-null  object
 3   cod_muni      588117 non-null  object
 4   municipio     588117 non-null  object
 5   cantidad      588117 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 26.9+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad']
theft_from_people_MinDefensa = df[relevant_cols]
theft_from_people_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588117 entries, 0 to 588116
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   cod_muni      588117 non-null  object
 1   departamento  588117 non-null  object
 2   municipio     588117 non-null  object
 3   fecha_hecho   588117 non-null  object
 4   cantidad      588117 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 22.4+ MB


### Convertir columna fecha_hecho a tipo date

In [5]:
# Hacer una copia explícita del DataFrame
theft_from_people_MinDefensa_ = theft_from_people_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
theft_from_people_MinDefensa_['fecha_hecho'] = pd.to_datetime(theft_from_people_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
theft_from_people_MinDefensa_['fecha_hecho'] = theft_from_people_MinDefensa_['fecha_hecho'].dt.year

In [6]:
theft_from_people_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003],
      dtype=int32)

### Verificar valores nulos

In [ ]:
theft_from_people_MinDefensa_.isnull().sum()

cod_muni        0
departamento    0
municipio       0
fecha_hecho     0
cantidad        0
dtype: int64

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(theft_from_people_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['ANTIOQUIA' 'ATLANTICO' 'BOYACA' 'CALDAS' 'CAQUETA' 'CASANARE' 'CAUCA'
 'CESAR' 'CHOCO' 'CUNDINAMARCA' 'CORDOBA' 'LA GUAJIRA' 'HUILA'
 'BOGOTA D.C.' 'SANTANDER' 'VALLE DEL CAUCA' 'BOLIVAR'
 'NORTE DE SANTANDER' 'TOLIMA' 'NARIÑO' 'RISARALDA' 'MAGDALENA' 'META'
 'QUINDIO' 'SAN ANDRES ISLAS' 'SUCRE' 'GUAVIARE' 'PUTUMAYO' 'AMAZONAS'
 'ARAUCA' 'GUAINIA' 'VAUPES' 'VICHADA']

Categorías en la columna 'municipio':
['RIONEGRO' 'SAN PEDRO DE LOS MILAGROS' 'MONTEBELLO' ... 'PALMAR'
 'SAN EDUARDO' 'LA SALINA']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    theft_from_people_MinDefensa_[col] = theft_from_people_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


dept_mpio_code dept_name mupio_name
0          97001    VAUPES       MITU
1          97161    VAUPES     CARURU
2          97511    VAUPES      PACOA
3          97666    VAUPES    TARAIRA
4          97777    VAUPES  PAPUNAHUA

 -  Verificar la consistencia de la columna "cod_muni" en el df theft_from_people_MinDefensa_

In [ ]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
theft_from_people_MinDefensa_['cod_muni'] = theft_from_people_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = theft_from_people_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [ ]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [ ]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

cod_muni   count  percentage
0         5  588117         1.0

In [ ]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = theft_from_people_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    05615
1    05664
2    05467
3    08638
4    08421



Nota: el 100%  de los codigos que identifican a los municipios en el df theft_from_people_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [ ]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(theft_from_people_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en theft_from_people_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en theft_from_people_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'SAN ANDRES ISLAS'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES ISLAS': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe theft_from_people_MinDefensa_
theft_from_people_MinDefensa_['departamento'] = theft_from_people_MinDefensa_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(theft_from_people_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en theft_from_people_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en theft_from_people_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(theft_from_people_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en theft_from_people_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en theft_from_people_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_theft_from_people_MinDefensa = theft_from_people_MinDefensa_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_theft_from_people_MinDefensa['source_id'] = 20

In [ ]:
final_theft_from_people_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'cantidad', 'source_id'], dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_theft_from_people_MinDefensa.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_theft_from_people_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588117 entries, 0 to 588116
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   dane_code         588117 non-null  object
 1   year_of_incident  588117 non-null  int32 
 2   amount            588117 non-null  int64 
 3   source_id         588117 non-null  int64 
dtypes: int32(1), int64(2), object(1)
memory usage: 15.7+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_theft_from_people_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/theft_from_people_MinDefensa.csv', index=False)